In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.optimizers import Adam
import keras
from keras.models import Sequential, Model
from keras.layers import *
from keras.utils import Sequence
from keras.layers import Conv2D, MaxPooling2D
from qkeras import *

from keras.utils import Sequence
from keras.callbacks import CSVLogger
from keras.callbacks import EarlyStopping

import os
import shutil
import random
import psutil

pi = 3.14159265359

maxval=1e9
minval=1e-9

2024-12-04 00:00:02.655899: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-04 00:00:03.730301: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
from dataloaders.OptimizedDataGenerator import OptimizedDataGenerator
from loss import *
from models.models import *

#### Scaling Lists for Different Pixel Pitches (dataset_2s):
* 100x25x100 um:  [150.0, 37.5, 10.0, 1.22]
* 50x25x100 um:   [75.0, 37.5, 10.0, 1.22]
* 50x20x100 um:   [75.0, 30.0, 10.0, 1.22]
* 50x15x100 um:   [75.0, 22.5, 10.0, 1.22]
* 50x12.5x100 um: [75.0, 18.75, 10.0, 1.22]
* 50x10x100 um:   [75.0, 15.0, 10.0, 1.22]

#### Scaling Lists for Different Pixel Pitches (dataset_3sr):
* 100x25x100 um:  [150.0, 37.5, 10.0, 10.0]
* 50x25x100 um:   [75.0, 37.5, 10.0, 10.0]
* 50x20x100 um:   [75.0, 30.0, 10.0, 10.0]
* 50x15x100 um:   [75.0, 22.5, 10.0, 10.0]
* 50x12.5x100 um: [75.0, 18.75, 10.0, 10.0]
* 50x10x100 um:   [75.0, 15.0, 10.0, 10.0]

#### Scaling Lists for Different Pixel Pitches (dataset_3sr, |cot$\beta$| $\leq$ 1.5 preselection):
* 100x25x100 um:  [150.0, 37.5, 10.0, 1.5], 
* 50x25x100 um:   [75.0, 37.5, 10.0, 1.5],
* 50x20x100 um:   [75.0, 30.0, 10.0, 1.5]
* 50x15x100 um:   [75.0, 22.5, 10.0, 1.5]
* 50x12.5x100 um: [75.0, 18.75, 10.0, 1.5]
* 50x10x100 um:   [75.0, 15.0, 10.0, 1.5]

In [3]:
batch_size = 5000
val_batch_size = 5000
train_file_size = 70
val_file_size = 10

# Specify what directory to save the tfrecords
tfrecords_dir_train = '/data/dajiang/smartPixels/tfrecords/tfrecords_dataset_3sr_100x25x150_20t_bs5000_cotBeta1P5_train'
tfrecords_dir_val = '/data/dajiang/smartPixels/tfrecords/tfrecords_dataset_3sr_100x25x150_20t_bs5000_cotBeta1P5_val'

training_generator = OptimizedDataGenerator(
    data_directory_path = '/data/dajiang/smartPixels/dataset_3s/dataset_3sr_100x25x150_parquets/unflipped/recon3D_cotBeta1P5/',
    labels_directory_path = '/data/dajiang/smartPixels/dataset_3s/dataset_3sr_100x25x150_parquets/unflipped/labels_cotBeta1P5/',
    is_directory_recursive = False,
    file_type = 'parquet',
    data_format = '3D',
    batch_size = batch_size,
    file_count = train_file_size,
    to_standardize= True,
    include_y_local= False,
    labels_list = ['x-midplane','y-midplane','cotAlpha','cotBeta'],
    scaling_list = [150.0, 37.5, 10.0, 1.5],
    input_shape = (20,13,21),
    transpose = (0,2,3,1),
    files_from_end = True,
    shuffle = True,

    #load_from_tfrecords_dir = '/data/dajiang/smartPixels/tfrecords/tfrecords_dataset_3sr_100x25x150_20t_bs5000_train',
    tfrecords_dir = tfrecords_dir_train,
    use_time_stamps = -1, #-1
    max_workers = 1, # Don't make this too large (will use up all RAM)
    seed = 10,
    quantize = True # Quantization ON
)

validation_generator = OptimizedDataGenerator(
    data_directory_path = '/data/dajiang/smartPixels/dataset_3s/dataset_3sr_100x25x150_parquets/unflipped/recon3D_cotBeta1P5/',
    labels_directory_path = '/data/dajiang/smartPixels/dataset_3s/dataset_3sr_100x25x150_parquets/unflipped/labels_cotBeta1P5/',
    is_directory_recursive = False,
    file_type = 'parquet',
    data_format = '3D',
    batch_size = val_batch_size,
    file_count = val_file_size,
    to_standardize= True,
    include_y_local= False,
    labels_list = ['x-midplane','y-midplane','cotAlpha','cotBeta'],
    scaling_list = [150.0, 37.5, 10.0, 1.5],
    input_shape = (20,13,21),
    transpose = (0,2,3,1),
    files_from_end = False,
    shuffle = True,

    #load_from_tfrecords_dir = '/data/dajiang/smartPixels/tfrecords/tfrecords_dataset_3sr_100x25x150_20t_bs5000_val',
    tfrecords_dir = tfrecords_dir_val,
    use_time_stamps = -1, #-1
    max_workers = 1, # Don't make this too large (will use up all RAM)
    seed = 10,
    quantize = True # Quantization ON
)

Directory /data/dajiang/smartPixels/tfrecords/tfrecords_dataset_3sr_100x25x150_20t_bs5000_cotBeta1P5_train does not exist and cannot be removed.


Saving batches as TFRecords: 100%|██████████| 158/158 [05:05<00:00,  1.94s/it]


Directory /data/dajiang/smartPixels/tfrecords/tfrecords_dataset_3sr_100x25x150_20t_bs5000_cotBeta1P5_val does not exist and cannot be removed.


Saving batches as TFRecords: 100%|██████████| 22/22 [00:46<00:00,  2.10s/it]


In [4]:
# compiles model
model=CreateModel((13,21,20),n_filters=5,pool_size=3)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 13, 21, 20)]      0         
                                                                 
 q_separable_conv2d (QSepar  (None, 11, 19, 5)         285       
 ableConv2D)                                                     
                                                                 
 q_activation (QActivation)  (None, 11, 19, 5)         0         
                                                                 
 q_conv2d (QConv2D)          (None, 11, 19, 5)         30        
                                                                 
 q_activation_1 (QActivatio  (None, 11, 19, 5)         0         
 n)                                                              
                                                                 
 average_pooling2d (Average  (None, 3, 6, 5)           0     

In [5]:
model.compile(optimizer=Adam(learning_rate=0.001), loss=custom_loss)

In [ ]:
# training
pitch = '100x25x150'
date = '3Dec2024'
base_dir = '/home/dajiang/smart-pixels-ml/weights/weights_7pitches/dataset_3sr_cotBeta1P5_weights/weights-{}-bs{}-{}-checkpoints'.format(pitch, batch_size, date)
 
os.mkdir(base_dir)
checkpoint_filepath = base_dir + '/weights.{epoch:02d}-t{loss:.2f}-v{val_loss:.2f}.hdf5'
mcp = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    save_best_only=False,
)

history = model.fit(x=training_generator,
                    validation_data=validation_generator,
                    callbacks=[mcp],
                    epochs=300,
                    shuffle=False, # shuffling now occurs within the data-loader
                    verbose=0)

2024-12-04 00:19:20.054184: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8906
2024-12-04 00:19:20.155048: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-12-04 00:19:20.455498: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-12-04 00:19:20.463536: I tensorflow/core/util/cuda_solvers.cc:179] Creating GpuSolver handles for stream 0x7fe8399cf2c0
2024-12-04 00:19:20.487439: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fd22cb3f190 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-12-04 00:19:20.487473: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-SXM4-40GB MIG 1g.5gb, Compute Capability 8.0
2024-12-04 00:19:20.492663: I tensorflow/compiler/mlir/tensorflow/utils/dump